# 사람 추가 ?

In [ ]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx

import seaborn as sns 
import matplotlib.pyplot as plt
plt.rc('font', family = 'NanumBarunGothic')
%matplotlib inline
from IPython.display import set_matplotlib_formats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 사람/투석

In [ ]:
df_node1 = pd.read_csv('/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_nodes.csv', encoding = 'cp949')
df_link1 = pd.read_csv('/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_links.csv', encoding = 'cp949')

In [ ]:
links1 = df_link1[['Source', 'Target', 'Weight']]
links1.tail()

,Source,Target,Weight
221,40.0,66.0,4.517606
222,40.0,94.0,7.889065
223,48.0,77.0,4.020387
224,48.0,79.0,3.713404
225,49.0,68.0,7.577889


In [ ]:
nodes1 = df_node1[['id', 'NAME', 'latitude', 'longitude','TYPE']]
nodes1.head()

,id,NAME,latitude,longitude,TYPE
0,0,환자,36.698989,127.481918,사람
1,1,환자,36.663737,127.417545,사람
2,2,환자,36.589344,127.487583,사람
3,3,환자,36.616492,127.519512,사람
4,4,환자,36.613333,127.540307,사람


### 네트워크 구성

In [ ]:
G1 = nx.Graph()
R = 6371e3

In [ ]:
for idx, row in nodes1.iterrows():
  G1.add_node(row['id'], Label=row['NAME'], latitude=row['latitude'], longitude=row['longitude'])

In [ ]:
for idx, row in links1.iterrows():
  G1.add_edge(row['Source'], row['Target'], weight = row['Weight'])

In [ ]:
print(nx.info(G1))

Graph with 96 nodes and 226 edges


### Folium

In [ ]:
std_point = tuple(nodes1.head(1)[['latitude', 'longitude']].iloc[0])
std_point

(36.6989887, 127.481918)

### 노드 표시

In [ ]:
map_osm = folium.Map(location=std_point, zoom_start=8)

for ix, row in nodes1.iterrows():
  location = (row['latitude'], row['longitude'])
  if row['NAME'] == '환자' :
    color = 'red'
  else :
    color = 'blue'
  folium.Circle(
      location = location,
      radius = 10*100,
      color = 'white',
      weight = 1,
      fill_opacity = 0.6,
      opacity = 1,
      fill_color = color ,
      fill = True 
  ).add_to(map_osm)

map_osm

In [ ]:
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in links1.iterrows():
  start = tuple(nodes1[nodes1['id'] == row['Source']][['latitude', 'longitude']].iloc[0])
  end = tuple(nodes1[nodes1['id'] == row['Target']][['latitude', 'longitude']].iloc[0])
  folium.PolyLine(
      locations = [start, end],
      color = 'black',
      line_cap = 'round',
      **kw,
  ).add_to(map_osm)
map_osm

In [ ]:
#노드끼리 모두 연결 되어 있는지 확인한 결과 False(연결되어 있지 않음)로 확인되었다.
nx.is_connected(G1)

False

In [ ]:
#나뉘어 있는 네트워크의 개수를 확인한 결과로 140 개의 네트워크로 구성됨을 확인하였다.
nx.number_connected_components(G1)

25

In [ ]:
[c for c in sorted(nx.connected_components(G1), key=len, reverse=True)]

[{0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  52.0,
  58.0,
  61.0,
  62.0,
  67.0,
  69.0,
  71.0,
  73.0,
  81.0,
  82.0,
  84.0,
  86.0,
  88.0,
  89.0,
  90.0,
  92.0,
  93.0,
  95.0},
 {9, 11.0, 12.0, 13.0, 14.0, 56.0, 60.0, 65.0, 70.0, 80.0, 83.0, 85.0, 87.0},
 {15, 16.0, 17.0, 19.0, 20.0, 53.0, 55.0, 63.0, 64.0, 74.0, 76.0},
 {21, 22.0, 23.0, 24.0, 91.0},
 {33, 34.0, 48.0, 77.0, 79.0},
 {25, 26.0, 27.0, 78.0},
 {28, 29.0, 30.0, 57.0},
 {39, 40.0, 66.0, 94.0},
 {31, 32.0, 59.0},
 {35, 36.0, 75.0},
 {38, 54.0},
 {49, 68.0},
 {10},
 {18},
 {37},
 {41},
 {42},
 {43},
 {44},
 {45},
 {46},
 {47},
 {50},
 {51},
 {72}]

In [ ]:
[nx.diameter(G1.subgraph(x)) for x in nx.connected_components(G1)]

[3, 3, 0, 4, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [ ]:
[nx.density(G1.subgraph(x)) for x in nx.connected_components(G1)]

[0.41595441595441596,
 0.3974358974358974,
 0,
 0.41818181818181815,
 0,
 0.4,
 0.5,
 0.5,
 0.6666666666666666,
 0.6,
 0.6666666666666666,
 0,
 1.0,
 0.6666666666666666,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 0,
 0,
 0]

In [ ]:
[nx.transitivity(G1.subgraph(x)) for x in nx.connected_components(G1)]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
#print('diamter:', nx.diameter(G))
diameter = max([max(j.values()) for (i,j) in nx.shortest_path_length(G1)])
print(' Graph Stucture')
print(' diameter : ',diameter )
print(' density:', nx.density(G1))
print(' transitivity:', nx.transitivity(G1))

 Graph Stucture
 diameter :  4
 density: 0.04956140350877193
 transitivity: 0


In [ ]:
 # 각 node별 centrality 값을 dictionary 형태로 보여줌
deg_cen = nx.degree_centrality(G1)
bet_cen = nx.betweenness_centrality(G1)
clo_cen = nx.closeness_centrality(G1)
eig_cen = nx.eigenvector_centrality(G1, max_iter=600) # max iter 의 deafault 값인 100으로 할 때 반복문이 끝나지 않아 600으로 다시 설정해줌
pagerank_cen = nx.pagerank(G1)
'''
katz_cen = nx.katz_centrality(G, max_iter=10000)
max_iter default값이 1000인데 10배인 10000으로 했는데도 아래와 같은 오류가 발생함
(-> 반복을 돌고 있거나 미세하게 계속 움직이고 있는 것으로 추정)
PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 10000 iterations')
'''

"\nkatz_cen = nx.katz_centrality(G, max_iter=10000)\nmax_iter default값이 1000인데 10배인 10000으로 했는데도 아래와 같은 오류가 발생함\n(-> 반복을 돌고 있거나 미세하게 계속 움직이고 있는 것으로 추정)\nPowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 10000 iterations')\n"

In [ ]:
# 각 centrality별 가장 값이 높은 4개의 node 추출
print('degree centrality:\n', sorted(deg_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
print('eigenvector centrality:\n', sorted(eig_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
# print('katz_centrality:\n', sorted(katz_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
print('pagerank centrality:\n', sorted(pagerank_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
print('betweenness centrality:\n', sorted(bet_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
print('closeness centrality:\n', sorted(clo_cen.items(), key=lambda x:x[1], reverse=True)[0:4])

degree centrality:
 [(0, 0.18947368421052632), (2, 0.18947368421052632), (5, 0.18947368421052632), (6, 0.18947368421052632)]
eigenvector centrality:
 [(0, 0.25671811852142373), (2, 0.25671811852142373), (5, 0.25671811852142373), (6, 0.25671811852142373)]
pagerank centrality:
 [(91, 0.027994688898668095), (9, 0.02535076114007002), (0, 0.02279486584623841), (57, 0.022587002722714182)]
betweenness centrality:
 [(0, 0.005111294673037634), (2, 0.005111294673037634), (5, 0.005111294673037634), (6, 0.005111294673037634)]
closeness centrality:
 [(0, 0.20928792569659443), (2, 0.20928792569659443), (5, 0.20928792569659443), (6, 0.20928792569659443)]


# 사람/보건소/병원

In [ ]:
df_node2 = pd.read_csv('/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_보건소_nodes.csv', encoding = 'cp949')
df_link2 = pd.read_csv('/content/drive/Shareddrives/2022데이터마이닝응용/4 프로젝트/코드/종연_코드/투석_최종/투석_사람_보건소_links.csv', encoding = 'cp949')

In [ ]:
links2 = df_link2[['Source', 'Target', 'Weight']]
links2.tail()

,Source,Target,Weight
1225,356.0,89.0,4.769601
1226,356.0,90.0,6.077417
1227,356.0,92.0,4.758572
1228,356.0,93.0,7.723483
1229,356.0,95.0,5.806746


In [ ]:
nodes2 = df_node2[['id', 'NAME', 'latitude', 'longitude','TYPE']]
nodes2.head()
nodes2['TYPE'].value_counts()

보건소    268
사람      52
병원      44
Name: TYPE, dtype: int64

### 네트워크 구성

In [ ]:
G2 = nx.Graph()
R = 6371e3

In [ ]:
for idx, row in nodes2.iterrows():
  G2.add_node(row['id'], Label=row['NAME'], latitude=row['latitude'], longitude=row['longitude'])

In [ ]:
for idx, row in links2.iterrows():
  G2.add_edge(row['Source'], row['Target'], weight = row['Weight'])

In [ ]:
print(G2)

Graph with 364 nodes and 1229 edges


### Folium

In [ ]:
std_point = tuple(nodes1.head(1)[['latitude', 'longitude']].iloc[0])
std_point

(36.6989887, 127.481918)

### 노드표시

In [ ]:
map_osm = folium.Map(location=std_point, zoom_start=8)

for ix, row in nodes2.iterrows():
  location = (row['latitude'], row['longitude'])
  if row['TYPE'] == '사람' :
    color = 'red'
  elif row['TYPE'] == '병원':
    color = 'green'
  else : 
    color = 'blue'
  folium.Circle(
      location = location,
      radius = 10*100,
      color = 'white',
      weight = 1,
      fill_opacity = 0.6,
      opacity = 1,
      fill_color = color ,
      fill = True 
  ).add_to(map_osm)

map_osm

In [ ]:
kw = {'opacity': 0.5, 'weight': 2}
for ix, row in links2.iterrows():
  start = tuple(nodes2[nodes2['id'] == row['Source']][['latitude', 'longitude']].iloc[0])
  end = tuple(nodes2[nodes2['id'] == row['Target']][['latitude', 'longitude']].iloc[0])
  folium.PolyLine(
      locations = [start, end],
      color = 'black',
      line_cap = 'round',
      **kw,
  ).add_to(map_osm)
map_osm

In [ ]:
#노드끼리 모두 연결 되어 있는지 확인한 결과 False(연결되어 있지 않음)로 확인되었다.
nx.is_connected(G2)

False

In [ ]:
#나뉘어 있는 네트워크의 개수를 확인한 결과로 140 개의 네트워크로 구성됨을 확인하였다.
nx.number_connected_components(G2)

159

In [ ]:
[c for c in sorted(nx.connected_components(G2), key=len, reverse=True)]

[{0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0,
  22.0,
  23.0,
  24.0,
  25.0,
  26.0,
  27.0,
  28.0,
  29.0,
  30.0,
  31.0,
  32.0,
  33.0,
  34.0,
  35.0,
  36.0,
  37.0,
  39.0,
  40.0,
  41.0,
  42.0,
  43.0,
  44.0,
  45.0,
  46.0,
  47.0,
  48.0,
  49.0,
  51.0,
  52.0,
  53.0,
  55.0,
  56.0,
  57.0,
  58.0,
  59.0,
  60.0,
  61.0,
  62.0,
  63.0,
  64.0,
  65.0,
  66.0,
  67.0,
  68.0,
  69.0,
  70.0,
  71.0,
  72.0,
  73.0,
  74.0,
  75.0,
  76.0,
  77.0,
  78.0,
  79.0,
  80.0,
  81.0,
  82.0,
  83.0,
  84.0,
  85.0,
  86.0,
  87.0,
  88.0,
  89.0,
  90.0,
  91.0,
  92.0,
  93.0,
  94.0,
  95.0,
  96.0,
  98.0,
  99.0,
  100.0,
  101.0,
  102.0,
  103.0,
  104.0,
  105.0,
  106.0,
  107.0,
  108.0,
  109.0,
  110.0,
  115.0,
  117.0,
  119.0,
  129.0,
  133.0,
  134.0,
  135.0,
  136.0,
  140.0,
  142.0,
  143.0,
  145.0,
  154.0,
  157.0,
  159.0,
  162.0,
 

In [ ]:
[nx.diameter(G2.subgraph(x)) for x in nx.connected_components(G2)]

[11,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
[nx.density(G2.subgraph(x)) for x in nx.connected_components(G2)]

[0.06172275519009188,
 0.4642857142857143,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
[nx.transitivity(G2.subgraph(x)) for x in nx.connected_components(G2)]

[0.10796059637912674,
 0.375,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
#print('diamter:', nx.diameter(G))
diameter = max([max(j.values()) for (i,j) in nx.shortest_path_length(G2)])
print(' Graph Stucture')
print(' diameter : ',diameter )
print(' density:', nx.density(G2))
print(' transitivity:', nx.transitivity(G2))

 Graph Stucture
 diameter :  11
 density: 0.01860260951170042
 transitivity: 0.10852816153028692


In [ ]:
# 각 centrality별 가장 값이 높은 4개의 node 추출
print('degree centrality:\n', sorted(deg_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
print('eigenvector centrality:\n', sorted(eig_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
# print('katz_centrality:\n', sorted(katz_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
print('pagerank centrality:\n', sorted(pagerank_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
print('betweenness centrality:\n', sorted(bet_cen.items(), key=lambda x:x[1], reverse=True)[0:4])
print('closeness centrality:\n', sorted(clo_cen.items(), key=lambda x:x[1], reverse=True)[0:4])

degree centrality:
 [(0, 0.18947368421052632), (2, 0.18947368421052632), (5, 0.18947368421052632), (6, 0.18947368421052632)]
eigenvector centrality:
 [(0, 0.25671811852142373), (2, 0.25671811852142373), (5, 0.25671811852142373), (6, 0.25671811852142373)]
pagerank centrality:
 [(91, 0.027994688898668095), (9, 0.02535076114007002), (0, 0.02279486584623841), (57, 0.022587002722714182)]
betweenness centrality:
 [(0, 0.005111294673037634), (2, 0.005111294673037634), (5, 0.005111294673037634), (6, 0.005111294673037634)]
closeness centrality:
 [(0, 0.20928792569659443), (2, 0.20928792569659443), (5, 0.20928792569659443), (6, 0.20928792569659443)]
